# Example: Writing a gas network model to the extended 3DCityDB

This example shows how to write a simple gas network model to the extended *3DCityDB*, using the `dblayer` package. Basic knowledge about [CityGML](http://www.citygmlwiki.org) and the [Utility Network ADE](https://en.wiki.utilitynetworks.sig3d.org) are required for this notebook. A nice overview of the of the basic concepts of the Utility Network ADE can be found [here](https://en.wiki.utilitynetworks.sig3d.org/images/upload/20160913_-_Kolbe_-_Modeling_3D_Utility_Networks_%2B_UtilityNetwork_ADE_Core_Model.pdf).

For the example, a simple gas network is used, as shown in the following figure.

<img src="./figures/gas_network.svg" style="width: 650px;">

## Getting started ...

Import the required funtionality from package `dblayer`.

In [1]:
from dblayer.helpers.utn.gas_network import *

Load data from CSV files, containing information about network nodes and pipes.

In [2]:
import pandas as pd
import os.path

nodes_csv_data = pd.read_csv( os.path.join( 'csv', 'gas_node_data.csv' ), index_col = 0 )
pipes_csv_data = pd.read_csv( os.path.join( 'csv', 'gas_pipe_data.csv' ), index_col = 0 )

Dict for most relevant data of network nodes (i.e., connections between pipes and network features) added to the database 

In [3]:
nodes_data = {}

## Connect to database

Define connection parameters for 3DCityDB instance.

In [4]:
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

Connect to database using an instance of class `DBAccess`.

In [5]:
db_access = DBAccess()
db_access.connect_to_citydb( connect )

For the purpose of this notebook yoi might want to use an empty 3DCityDB. To this end, the next two lines erase all data from an existing database. **Only execute the next two lines if you really want to erase all data from your database!**

In [6]:
db_access.cleanup_citydb_schema()
db_access.cleanup_simpkg_schema()

## Write the network to the database
Define the spatial reference ID (SRID) of the network. This should be consistent with the setup of your database.

In [7]:
srid = 4326

Insert a new *network* and an associated *network graph*. The *network* basically contains the topographical representation - the so-called *network features* - of all the network network components. The *network graph* contains the toplogical representation - of all these components the topological links between them. Use function `write_network_to_db` from package `dblayer.helpers.utn.gas_network`.

In [8]:
( ntw_id, ntw_graph_id ) = write_network_to_db(
    db_access,
    name = 'gas_example_network',
    id = 3000
    )

Add nodes to the network that will be used as connection points between pipes and other network features. Use function `write_network_node_to_db` from package `dblayer.helpers.utn.gas_network`.

In [9]:
for node_name, node_csv_data in nodes_csv_data.iterrows():

    nodes_data[node_name] = write_network_node_to_db(
        db_access,
        node_name,
        node_csv_data.level,
        Point2D( node_csv_data.x, node_csv_data.y ),
        srid,
        ntw_id,
        ntw_graph_id )

Add all pipes to the network and connect them. Use function `write_round_pipe_to_db` from package `dblayer.helpers.utn.gas_network`.

In [10]:
for pipe_name, pipe_csv_data in pipes_csv_data.iterrows():

    write_round_pipe_to_db(
        db_access,
        pipe_name,
        nodes_data[pipe_csv_data.from_node],
        nodes_data[pipe_csv_data.to_node],
        srid,
        ntw_id,
        ntw_graph_id,
        int_diameter = pipe_csv_data.diameter_m,
        int_diameter_unit = 'm'
    )

Add gas sinks to the network and connect them. Use function `write_gas_sink_to_db` from package `dblayer.helpers.utn.gas_network`.

In [11]:
write_gas_sink_to_db(
    db_access,
    'sink-N2',
    nodes_data['node-N2'],
    10.,
    'kW',
    srid,
    ntw_id,
    ntw_graph_id 
    )

write_gas_sink_to_db(
    db_access,
    'sink-N3',
    nodes_data['node-N3'],
    15.,
    'kW',
    srid,
    ntw_id,
    ntw_graph_id 
    )

Add station to the network and connect it. Use function `write_station_to_db` from package `dblayer.helpers.utn.gas_network`.

In [12]:
write_station_to_db(
    db_access,
    'station-N0-N1',
    nodes_data['node-N0'],
    nodes_data['node-N1'],
    50.,
    0.025E5,
    srid,
    ntw_id,
    ntw_graph_id
    )

Add feeder to the network and connect it. Use function `write_feeder_to_db` from package `dblayer.helpers.utn.gas_network`.

In [13]:
write_feeder_to_db(
    db_access,
    'feeder-F',
    nodes_data['node-NF'],
    50.,
    0.9E5,
    srid,
    ntw_id,
    ntw_graph_id
    )

Commit changes to database.

In [14]:
db_access.commit_citydb_session()

Done.